# School analysis

This is an analysis of student performance in maths and reading in a specific local government area (LGA). The influence of the student's school on their performance is considered.


In [297]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

df.head()

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635


The following function will be used to appropriately format datatables below.

In [298]:

def formatted(df):
    
    def format_col(d,col,f):   
        if col in d.columns:
            d[col] = d[col].apply(lambda x: f.format(x))
        return d

    d = df.copy()
    
    pct = '{:.1%}'
    intf = '{:,}'
    num = '{:,.4}'
    curr = '${:,.2f}'

    format_col(d,'Total Students',intf)
    format_col(d,'Total School Budget',curr)
    format_col(d,'Per Student Budget',curr)
    format_col(d,'% Passing Maths',pct)
    format_col(d,'% Passing Reading',pct)
    format_col(d,'% Overall Passing',pct)
    format_col(d,'Average Maths Score',num)
    format_col(d,'Average Reading Score',num)

    return d

## Local Government Area Summary

A summary of the LGA that details the:

- Total number of unique schools
- Total students
- Total budget
- Average maths score
- Average reading score
- % passing maths (the percentage of students who passed maths)
- % passing reading (the percentage of students who passed reading)
- % overall passing (the percentage of students who passed maths AND reading)

In [299]:

summary = {
    'Total Schools': [df.school_name.nunique()],
    'Total Students': [len(df)], 
    'Total School Budget': [df.groupby('school_name').mean().budget.sum()],
    'Average Maths Score': [df.maths_score.mean()],
    'Average Reading Score': [df.reading_score.mean()],
    '% Passing Maths': [df[df['maths_score']>=50].shape[0]/len(df)],
    '% Passing Reading': [df[df['reading_score']>=50].shape[0]/len(df)],
    '% Overall Passing': [(df[(df['maths_score']>=50) & (df['reading_score']>=50)].shape[0])/len(df)]
}

area_summary = formatted(pd.DataFrame.from_dict(summary))

area_summary


/var/folders/mp/sc4qw2553bb18b1y2my327zc0000gn/T/ipykernel_66725/427220432.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  'Total School Budget': [df.groupby('school_name').mean().budget.sum()],


,Total Schools,Total Students,Total School Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.34,69.98,86.1%,84.4%,72.8%


## School Summary

The code below creates an overview table, `area_summary`, that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)


In [300]:
# function to apply across groups
def aggregate_schools2(d):
    school_type = d.type.iloc[0]
    total_students = d.student_name.count()
    school_budget = d.budget.iloc[0] # first instance
    avg_maths = d.maths_score.mean()
    avg_reading = d.reading_score.mean()
    n_maths = d[d['maths_score']>=50].shape[0] # nrow of filtered table
    n_reading = d[d['reading_score']>=50].shape[0]
    n_overall = d[(d['maths_score']>=50) & (d['reading_score']>=50)].shape[0]
    pct_maths = n_maths/total_students
    pct_reading = n_reading/total_students
    pct_overall = n_overall/total_students
    budget_per_student = school_budget/total_students

    labels = [
        'School Type',
        'Total Students',
        'Total School Budget',
        'Per Student Budget',
        'Average Maths Score',
        'Average Reading Score',
        '% Passing Maths',
        '% Passing Reading',
        '% Overall Passing'
    ]
    
    return pd.Series([
        school_type,
        total_students,
        school_budget,
        budget_per_student,
        avg_maths,
        avg_reading,
        pct_maths,
        pct_reading,
        pct_overall
    ], index=labels)

dff = df \
    .groupby('school_name') \
    .apply(aggregate_schools2) \
    .reset_index() \
    .rename(columns={'school_name':'School Name'})


per_school_summary = formatted(dff)

per_school_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.35,71.01,91.6%,87.4%,80.1%
1,Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.66,71.36,90.9%,89.1%,80.8%
2,Figueroa High School,Government,"2,949","$1,884,411.00",$639.00,68.7,69.08,81.7%,82.8%,67.7%
3,Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.09,69.57,82.4%,82.2%,67.5%
4,Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.79,71.25,91.2%,88.5%,81.3%
5,Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.87,69.19,80.9%,81.9%,66.4%
6,Holden High School,Independent,427,"$248,087.00",$581.00,72.58,71.66,89.9%,88.5%,78.9%
7,Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.94,68.91,81.7%,81.5%,66.7%
8,Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.84,69.04,82.1%,82.0%,67.2%
9,Pena High School,Independent,962,"$585,858.00",$609.00,72.09,71.61,91.7%,86.6%,79.2%



## Top Performing Schools (By % Overall Passing)

In [301]:
top_schools = formatted(dff.sort_values(by='% Overall Passing', ascending=False).head(5))

top_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
4,Griffin High School,Independent,"1,468","$917,500.00",$625.00,71.79,71.25,91.2%,88.5%,81.3%
1,Cabrera High School,Independent,"1,858","$1,081,356.00",$582.00,71.66,71.36,90.9%,89.1%,80.8%
0,Bailey High School,Government,"4,976","$3,124,928.00",$628.00,72.35,71.01,91.6%,87.4%,80.1%
14,Wright High School,Independent,"1,800","$1,049,400.00",$583.00,72.05,70.97,91.8%,86.7%,79.7%
10,Rodriguez High School,Government,"3,999","$2,547,363.00",$637.00,72.05,70.94,90.8%,87.4%,79.4%


## Bottom Performing Schools (By % Overall Passing)

In [302]:
bottom_schools = formatted(dff.sort_values(by='% Overall Passing').head(5))

bottom_schools

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
5,Hernandez High School,Government,"4,635","$3,022,020.00",$652.00,68.87,69.19,80.9%,81.9%,66.4%
7,Huang High School,Government,"2,917","$1,910,635.00",$655.00,68.94,68.91,81.7%,81.5%,66.7%
8,Johnson High School,Government,"4,761","$3,094,650.00",$650.00,68.84,69.04,82.1%,82.0%,67.2%
13,Wilson High School,Independent,"2,283","$1,319,574.00",$578.00,69.17,68.88,82.8%,81.3%,67.5%
3,Ford High School,Government,"2,739","$1,763,916.00",$644.00,69.09,69.57,82.4%,82.2%,67.5%


## Maths Scores by Year


In [303]:
maths_scores_by_year = df[['school_name','maths_score','year']] \
    .rename(columns={'school_name': "School Name", 'year': "School Year"}) \
    .groupby(['School Name','School Year']) \
    .mean() \
    .reset_index() \
    .pivot(index = 'School Name',columns = 'School Year',values = 'maths_score')

maths_scores_by_year

School Year,9,10,11,12
School Name,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## Reading Score by Year

In [304]:
reading_scores_by_year = df[['school_name','reading_score','year']] \
    .rename(columns={'school_name': "School Name", 'year': "School Year"}) \
    .groupby(['School Name','School Year']) \
    .mean() \
    .reset_index() \
    .pivot(index = 'School Name',columns = 'School Year',values = 'reading_score')

reading_scores_by_year

School Year,9,10,11,12
School Name,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

In [305]:
dff_spend = dff.copy()

spend_labels = ['<$585','$585-630','$630-645','$645-680']
spend_bins = [0,585,630,645,680]

dff_spend['Spending Ranges (Per Student)'] = pd.cut(dff_spend['Per Student Budget'], bins=spend_bins, labels=spend_labels)

dff_spend_out = dff_spend[[
    'Spending Ranges (Per Student)',
    'Average Maths Score',
    'Average Reading Score',
    '% Passing Maths',
    '% Passing Reading',
    '% Overall Passing'
    ]].groupby('Spending Ranges (Per Student)').mean()

spending_summary = formatted(dff_spend_out)

spending_summary


,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.8%,86.4%,76.7%
$585-630,72.07,71.03,91.5%,87.3%,79.9%
$630-645,69.85,69.84,84.7%,83.8%,71.0%
$645-680,68.88,69.05,81.6%,81.8%,66.8%


## Scores by School Size

In [306]:
dff_size = dff.copy()

size_labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']
size_bins = [0,1000,2000,5000]

dff_size['School Size'] = pd.cut(dff_size['Total Students'], bins = size_bins, labels = size_labels)

dff_size_out = dff_size[[
    'School Size',
    'Average Maths Score',
    'Average Reading Score',
    '% Passing Maths',
    '% Passing Reading',
    '% Overall Passing'
    ]].groupby('School Size').mean()

size_summary = formatted(dff_size_out)

size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.34,71.64,90.8%,87.6%,79.1%
Medium (1000-2000),71.42,70.72,89.8%,86.7%,78.0%
Large (2000-5000),69.75,69.58,84.3%,83.3%,70.3%


## Scores by School Type

In [307]:
dff_type = dff.copy()[[
    'School Type',
    'Average Maths Score',
    'Average Reading Score',
    '% Passing Maths',
    '% Passing Reading',
    '% Overall Passing'
    ]].groupby('School Type').mean()

type_summary = formatted(dff_type)

type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.83,69.68,84.5%,83.6%,70.7%
Independent,71.37,70.72,89.2%,86.2%,77.0%


# Findings

The above analysis investigates the performance of schools in a specific LGA. Performance is assessed across student scores in Maths and Reading, from which dummy values (pass/fail) are also computed, enabling schools to be compared by the proportion of students passing a specified score threshold (50).

Examination of scores by grade level across different schools did not indicate that grade level, or associated cohort effects, exerted a significant effect on the student scores. However, the school itself does have an effect on the score.

The nature of the school effect is further investigated through analysis of specific school characterisitics: school size, spending per student, and the type of school (government or independent).

Higher spends per student was not directly correlated with student scores, indeed schools with lower per-student budgets ($585-630) had the highest scores.

A direct effect was observed for school size and type, however, with smaller schools and independent schools recording higher scores.



